In [ ]:
from utils import setrootdir
setrootdir("ppgcc-coautorias")

In [ ]:
import os

from dotenv import load_dotenv

from src.preprocessing import Preprocesser

In [ ]:
load_dotenv()

DATASET_DIRECTORY = os.getenv("DATASET_DIRECTORY")
METADATA_FILE = os.getenv("METADATA_FILE")

# 2. Preprocessing

In [ ]:
preprocesser = Preprocesser(
    data_dir=DATASET_DIRECTORY,
    metadata_file=METADATA_FILE
)

## 2.1. DataFrames construction

In [ ]:
institution_registry = preprocesser.extract_institution_registry(
    selection_dir_name="01-selection"
)

institution_registry

### 2.1.1. Productions

In [ ]:
df_productions = preprocesser.frame_productions_all_institutions(institution_registry)
df_productions

## 2.2. Productions with no authors

In [ ]:
df_productions = preprocesser.drop_authorless_productions(df_productions)
df_productions

## 2.3. Citation normalization

In [ ]:
df_productions = preprocesser.normalize_authors(df_productions)
df_productions

In [ ]:
df_productions = preprocesser.explode_authors(df_productions)
df_productions

In [ ]:
preprocesser.write_parquet(df_productions, step="02-preprocessing", name="productions")